并发：不同的代码块交替执行

并行：不同的代码块同时执行

# 秒杀系统

### 系统要求

高性能：秒杀涉及大量的并发读和并发写，因此要求性能必须高；

一致性：秒杀中商品减库存的实现方式同样关键。可想而知，有限数量的商品同一时刻被很多倍的请求同时来减库存。在大并发更新数据的过程中保证数据的准确性，难度可想而知；

高可用：出现问题的时候要保证可用；

# 架构原则

1. 数据要尽量少

   所谓数据尽量少，就是指用户请求的数据能少则少。请求的数据包括上传给系统的数据和系统返回给用户的数据（通常就是网页）。
   
   因为这些数据在网络上传输需要时间，其次不管是请求数据还是返回数据都需要服务器来处理，而服务器在写网络时都要做压缩和编码，这些都非常消耗CPU，所以减少传输的数据量可以显著的减少CPU的使用。例如，我们可以简化秒杀页面的大小，去掉不必要的页面装修效果等。
   
   其次，“数据要尽量少”还要求系统依赖的数据能少就少，包括系统完成某些业务逻辑需要读取和保存的数据，这些数据一般是和后台服务和数据库打交道的。调用其他服务会涉及到数据的序列化和反序列化，这也是CPU的一大杀手，同样也会增加延时。而且，数据库本身也容易成为一个瓶颈，所以和数据库打交道越少越好，越简单越好。
   

2. 请求数要尽量少

    用户请求的页面返回后，浏览器渲染页面还包含其他的额外请求，比如说这个页面依赖的CSS/javascript，图片以及Ajax请求等，这些额外的请求应该尽量少。应为浏览器没发出一个请求都多少会有一些消耗，例如建立连接要做三次握手，另外不同请求访问的域名不一样，还需要做DNS域名解析，可能会耗时更久。
    
    
3. 路径要尽量短

    所谓“路径”，就是用户发出请求到返回数据这个过程中，需求经过的中间的节点数。
    

4. 依赖要尽量少

    所谓依赖，指的是要完成一次用户请求必须依赖的系统或者服务，这里的依赖指的是强依赖。


5. 不要有单点
    
    系统中的单点可以说是系统架构上的一个大忌，因为单点意味着没有备份，风险不可控，我们设计分布式系统最重要的原则就是“消除单点”。
    

# 做好动静分离

### 何为动静数据

简单来说，“静态数据”和“动态数据”的主要区别就是看页面中输出的数据是否和URL、浏览器、时间、地域相关，以及是否含有Cookie等私密信息。

### 如何做动静分离的改造

下面我们从5个方面来分离动态内容：

1. URL唯一化：商品详情系统天然地就可以做到URL唯一化，比如每个商品都由ID来标识，那么http://item.xxx.com/item.htm?id=xxx 就可以作为唯一的URL标识。为啥要URL唯一呢？前面说了我们是要缓存整个HTTP连接，那么以什么作为Key呢？就以URL作为缓存的Key，例如id=xxx 这个格式进行区分。

2. 分离浏览者相关的因素：浏览者相关的因素包括是否已登录，以及登录身份等，这些因素可以单独拆出来，通过动态请求来获取。

3. 分离时间因素：服务器输出的时间也要动态请求来获取。

4. 异步化地域因素：详情页上与地域相关的因素做成异步获取。

5. 服务端输出的页面包含的Cookie可以通过代码软件来删除：如，web服务器Varnish可以通过

    `unset req.http.cookie`
    
    命令去掉Cookie。注意，这里说的去掉Cookie并不是用户端收到的页面不含Cookie了，而是说，在缓存的静态数据中不含有Cookie。